# Mount the Google Drive. Import all the necessary Libraries

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime
from sklearn.model_selection import train_test_split
import os

print("tensorflow version : ", tf.__version__)
print("tensorflow_hub version : ", hub.__version__)

tensorflow version :  1.15.2
tensorflow_hub version :  0.9.0


In [ ]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
    print('We will use the GPU:', device_name)
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0
We will use the GPU: /device:GPU:0


In [ ]:
# Set the output directory for saving model file
OUTPUT_DIR = '/GD/My Drive/Colab Notebooks/BERT/bert_imbalanced'

#@markdown Whether or not to clear/delete the directory and create a new one
DO_DELETE = False #@param {type:"boolean"}

if DO_DELETE:
  try:
    tf.gfile.DeleteRecursively(OUTPUT_DIR)
  except:
    pass

tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))

***** Model output directory: /GD/My Drive/Colab Notebooks/BERT/bert_imbalanced *****


# Read the Training and Test Data

In [ ]:
train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/train_file.txt', sep='{}{}{}', engine = 'python')
test = pd.read_csv("/content/drive/My Drive/Colab Notebooks/test_file.txt", sep= '{}{}{}', engine = 'python')

#from sklearn.model_selection import train_test_split

train, val =  train,test

In [ ]:
train.head()

,Label,Text
0,__label__ob,"Friends, are we ready to stop pretending that ..."
1,__label__ob,A new set of Ten Commandments for 2009 There ...
2,__label__ob,Post navigation What gets you on the go? Incr...
3,__label__ob,Aaronovitch Takes On The BNP David Aaronovitc...
4,__label__ob,The Church of England's General Synod votes on...


# Label Encoding All the 26 Labels and Then Map Them

In [ ]:
label_encode = {}
for i, v  in enumerate(train['Label'].unique()):
  label_encode[v] = i
label_encode

{'__label__av': 20,
 '__label__df': 17,
 '__label__dp': 23,
 '__label__ds': 9,
 '__label__dt': 7,
 '__label__en': 21,
 '__label__fi': 22,
 '__label__fs': 18,
 '__label__ha': 15,
 '__label__ht': 25,
 '__label__ib': 5,
 '__label__it': 10,
 '__label__ne': 2,
 '__label__ob': 0,
 '__label__pb': 3,
 '__label__po': 14,
 '__label__qa': 11,
 '__label__ra': 8,
 '__label__re': 19,
 '__label__rs': 13,
 '__label__rv': 6,
 '__label__sl': 24,
 '__label__sr': 16,
 '__label__ss': 1,
 '__label__tb': 12,
 '__label__tv': 4}

In [ ]:
train['Label_enc'] = train['Label'].map(label_encode)
val['Label_enc'] = val['Label'].map(label_encode)
val.head()

,Label,Text,Label_enc
0,__label__ob,Just another UBC Blogs site Main menu Post n...,0
1,__label__ob,Why getting Australians into mining jobs doesn...,0
2,__label__ob,When people see the real Obama in his complete...,0
3,__label__ob,Could we have a Corporate-free Olympics? Yes w...,0
4,__label__ob,Is Obama trying to create a crisis? And if so...,0


In [ ]:
print("Training Set Shape :", train.shape)
print("Test Set Shape :", val.shape)

Training Set Shape : (17588, 3)
Test Set Shape : (2210, 3)


In [ ]:
train['Label_enc'].value_counts()

2     5577
16    1711
0     1445
17    1268
3     1203
7     1108
6      802
11     638
25     586
9      482
24     369
21     326
13     323
8      294
5      236
20     222
23     213
10     193
15     145
1      129
12      90
19      89
22      76
14      38
18      16
4        9
Name: Label_enc, dtype: int64

# Shuffle the Dataset

Shuffling helps model not to get biased and each datapoint creates independent change in the model

In [ ]:
from sklearn.utils import shuffle

train = shuffle(train)
train.head()

,Label,Text,Label_enc
6365,__label__ne,New report published by GPs into the effects o...,2
3560,__label__ne,A military judge said accused Fort Hood gunman...,2
17170,__label__ht,How to Deal With Social Media Marketing Pet Pe...,25
11719,__label__qa,Other Answers (4) depends on the situation. F...,11
6100,__label__ne,"Women spend 3,276 hours getting ready Women s...",2


# Running BERT

In [ ]:
#Installing BERT module
!pip install bert-tensorflow

#Importing BERT modules
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

     |████████████████████████████████| 71kB 3.1MB/s 


In [ ]:
DATA_COLUMN = 'Text'
LABEL_COLUMN = 'Label_enc'

train_InputExamples = train.apply(lambda x: bert.run_classifier.InputExample(guid=None,
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

val_InputExamples = val.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

In [ ]:
val.head()

,Label,Text,Label_enc
0,__label__ob,Just another UBC Blogs site Main menu Post n...,0
1,__label__ob,Why getting Australians into mining jobs doesn...,0
2,__label__ob,When people see the real Obama in his complete...,0
3,__label__ob,Could we have a Corporate-free Olympics? Yes w...,0
4,__label__ob,Is Obama trying to create a crisis? And if so...,0


In [ ]:
train_InputExamples

6365     <bert.run_classifier.InputExample object at 0x...
3560     <bert.run_classifier.InputExample object at 0x...
17170    <bert.run_classifier.InputExample object at 0x...
11719    <bert.run_classifier.InputExample object at 0x...
6100     <bert.run_classifier.InputExample object at 0x...
                               ...                        
8013     <bert.run_classifier.InputExample object at 0x...
16918    <bert.run_classifier.InputExample object at 0x...
4136     <bert.run_classifier.InputExample object at 0x...
794      <bert.run_classifier.InputExample object at 0x...
913      <bert.run_classifier.InputExample object at 0x...
Length: 17588, dtype: object

In [ ]:
print("Row 0 - guid of training set : ", train_InputExamples.iloc[0].guid)
print("\n__________\nRow 0 - text_a of training set : ", train_InputExamples.iloc[0].text_a)
print("\n__________\nRow 0 - text_b of training set : ", train_InputExamples.iloc[0].text_b)
print("\n__________\nRow 0 - label of training set : ", train_InputExamples.iloc[0].label)

Row 0 - guid of training set :  None

__________
Row 0 - text_a of training set :  New report published by GPs into the effects of current austerity measures  Concerns have been raised in several quarters about the consequences of the Government's welfare reforms and other austerity measures, which have been implemented since October 2010. These concerns include the negative impact that cuts in benefits are having on some of society's most vulnerable individuals and families. GPs and primary healthcare professionals are at the frontline in responding to the needs of these people.  A new report published by GPs,'GPs at the Deep End', comments on the effects of current austerity measures on the most deprived GP practices in Scotland. 'GPs at the Deep End' work in 100 general practices serving the most socio-economically deprived populations in Scotland. This report draws on the recent experiences of Deep End practices, as they were asked to reflect on the effects of austerity measures on

In [ ]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [ ]:
#Here is what the tokenised sample of the first training set observation looks like
print(tokenizer.tokenize(train_InputExamples.iloc[0].text_a))

UnparsedFlagAccessError: ignored

In [ ]:
label_list = train['Label_enc'].unique()
# We'll set sequences to be at most 128 tokens long.
MAX_SEQ_LENGTH = 128

# Convert our train and validation features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

val_features = bert.run_classifier.convert_examples_to_features(val_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

In [ ]:
val_InputExamples.shape

In [ ]:
train_InputExamples

In [ ]:
val.head(10)

In [ ]:
#Example on first observation in the training set
print("Sentence : ", train_InputExamples.iloc[0].text_a)
print("-"*30)
print("Tokens : ", tokenizer.tokenize(train_InputExamples.iloc[0].text_a))
print("-"*30)
print("Input IDs : ", train_features[0].input_ids)
print("-"*30)
print("Input Masks : ", train_features[0].input_mask)
print("-"*30)
print("Segment IDs : ", train_features[0].segment_ids)
print('\n\n')
print('Label ID:',train_features[-1].label_id)

In [ ]:

def create_model(is_predicting, input_ids, input_mask, segment_ids, labels, num_labels):    
    bert_module = hub.Module(BERT_MODEL_HUB, trainable=True)
    bert_inputs = dict(input_ids=input_ids, input_mask=input_mask, segment_ids=segment_ids)
    bert_outputs = bert_module(inputs=bert_inputs, signature="tokens", as_dict=True)
    output_layer = bert_outputs["pooled_output"] # sequence_outputs as another option
    hidden_size = output_layer.shape[-1].value

    # Create our own layer to tune for classification.
    output_weights = tf.get_variable("output_weights", [num_labels, hidden_size], initializer=tf.truncated_normal_initializer(stddev=0.02))

    output_bias = tf.get_variable("output_bias", [num_labels], initializer=tf.zeros_initializer())

    with tf.variable_scope("loss"):

        output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

        logits = tf.matmul(output_layer, output_weights, transpose_b=True)
        logits = tf.nn.bias_add(logits, output_bias)
        log_probs = tf.nn.log_softmax(logits, axis=-1)

        # Convert labels into one-hot encoding
        one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

        predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
        # If we're predicting, we want predicted labels and the probabiltiies.
        if is_predicting:
            return (predicted_labels, log_probs)

        # If we're train/eval, compute loss between predicted and actual label
        per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
        loss = tf.reduce_mean(per_example_loss)
        return (loss, predicted_labels, log_probs)

In [ ]:
#A function that adapts our model to work for training, evaluation, and prediction.

# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps, num_warmup_steps):
    
    """Returns `model_fn` closure for TPUEstimator."""
    def model_fn(features, labels, mode, params):
        # pylint: disable=unused-argument
        """The `model_fn` for TPUEstimator."""
        input_ids = features["input_ids"]
        input_mask = features["input_mask"]
        segment_ids = features["segment_ids"]
        label_ids = features["label_ids"]

        is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
        # TRAIN and EVAL. 
        if not is_predicting:
              (loss, predicted_labels, log_probs) = create_model(is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

              train_op = bert.optimization.create_optimizer(loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

              # Calculate evaluation metrics. 
              def metric_fn(label_ids, predicted_labels):

                    accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
                    auc = tf.metrics.auc(label_ids, predicted_labels)
                    predicted_labels = predicted_labels
                    # f1_score = tf.contrib.metrics.f1_score(label_ids, predicted_labels)
                    # recall = tf.metrics.recall(label_ids, predicted_labels)
                    # precision = tf.metrics.precision(label_ids, predicted_labels)
                    # true_pos = tf.metrics.true_positives(label_ids, predicted_labels)
                    # true_neg = tf.metrics.true_negatives(label_ids, predicted_labels)   
                    # false_pos = tf.metrics.false_positives(label_ids, predicted_labels)  
                    # false_neg = tf.metrics.false_negatives(label_ids,  predicted_labels)

                    return {
                        "eval_accuracy": accuracy,
                        #"f1_score":f1_score,
                        # "recall":recall,
                        # "precision":precision,
                        # "true_positives": true_pos,
                        # "true_negatives": true_neg,
                        # "false_positives": false_pos,
                        # "false_negatives": false_neg, 
                        #"predicted_labels": (predicted_labels)
                        }
                
              eval_metrics = metric_fn(label_ids, predicted_labels)

              if mode == tf.estimator.ModeKeys.TRAIN:
                      return tf.estimator.EstimatorSpec(mode=mode,loss=loss,train_op=train_op)
              else:
                    return tf.estimator.EstimatorSpec(mode=mode,  loss=loss, eval_metric_ops=eval_metrics)
        else:
            (predicted_labels, log_probs) = create_model(is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)
            predictions = {'probabilities': log_probs,'labels': predicted_labels}
            return tf.estimator.EstimatorSpec(mode, predictions=predictions)

    # Return the actual model function in the closure
    return model_fn

In [ ]:
# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 20
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 2.0
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS = 100

In [ ]:
# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

In [ ]:
# Specify outpit directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir='/content/drive/My Drive/Colab Notebooks/BERT/bert_imbalanced',
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [ ]:
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

#metric_fn = model_fn(features , labels , mode , params)

###**TensorFlow Estimators**
The Estimator object wraps a model which is specified by a model_fn, which, given inputs and a number of other parameters, returns the ops necessary to perform training, evaluation, or predictions.

In [ ]:
estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})

In [ ]:
# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

In [ ]:

len(train_features)

In [ ]:
%%time
print(f'Beginning Training!')
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)

In [ ]:
test_input_fn = run_classifier.input_fn_builder(
    features=val_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [ ]:
#evaluate on test dataset
estimator.evaluate(input_fn=test_input_fn, steps=None)

In [ ]:
preds_train = estimator.predict(input_fn=train_input_fn)
preds_train_label = []
for i in (preds_train):
  preds_train_label.append(i['labels'])
  #print(i)
  #break

In [ ]:
preds = estimator.predict(input_fn=test_input_fn)
pred_label = []
for i in  (preds):
  pred_label.append(i['labels'])
  # print(i['labels'])
  

In [ ]:
#print(pred_label)
print(val['Label_enc'])

In [ ]:
orig_label = val['Label_enc'].tolist()
print(orig_label)

In [ ]:
from sklearn.metrics import classification_report
y_true = orig_label
y_pred = pred_label
print(classification_report(y_true, y_pred))

In [ ]:
print(y_true)

In [ ]:
print(y_pred)